# Capítulo 07 — Instruction Tuning: Criando um Assistente

🎯 **Objetivos:** Transformar o modelo completador em um assistente útil usando **SFT (Supervised Fine-Tuning)**.

![SFT](./infograficos/04-pipeline-sft.png)

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os, sys
REPO_NAME = "fazendo-um-llm-do-zero"
if 'google.colab' in str(get_ipython()):
    if not os.path.exists(REPO_NAME):
        get_ipython().system(f"git clone https://github.com/vongrossi/{REPO_NAME}.git")
    if os.path.exists(REPO_NAME) and os.getcwd().split('/')[-1] != REPO_NAME:
        os.chdir(REPO_NAME)
if os.getcwd() not in sys.path: sys.path.append(os.getcwd())
print("📂 Diretório atual:", os.getcwd())

In [ ]:
import os, sys, torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"

# 📂 Carregamento do Checkpoint do Cap 05
if not os.path.exists("gpt_checkpoint.pt"):
    from google.colab import files
    print("📤 Por favor, suba o 'gpt_checkpoint.pt' gerado no Capítulo 05:")
    uploaded = files.upload()

ckpt = torch.load("gpt_checkpoint.pt", map_location=device, weights_only=False)
stoi, itos = ckpt['stoi'], ckpt['itos']
config = ckpt['config']
context_size = config.context_size

# Encoder: mapeia caracteres desconhecidos para espaço (evita colapso do prompt)
def encode(s):
    res = []
    unk_id = stoi.get(' ', 0)
    for c in s.lower():
        res.append(stoi.get(c, unk_id))
    return res

decode = lambda l: ''.join([itos[i] for i in l])

print(f"🧠 Modelo pré-treinado carregado!")
print(f"📏 Contexto: {context_size} | Vocabulário: {len(stoi)} caracteres")
if '#' not in stoi: print("⚠️ AVISO: Seu checkpoint não possui o caractere '#'. Re-execute o Capítulo 05 com o novo dataset.")


## 1. Dataset de Instruções

Criamos pares de Pergunta e Resposta para o alinhamento.

In [ ]:
instructions = [
    {"q": "o que o gato fez?", "a": "o gato subiu no telhado e pulou o muro."},
    {"q": "onde o cachorro dormiu?", "a": "o cachorro dormiu no sofa e no tapete."},
    {"q": "defina inteligencia artificial", "a": "inteligencia artificial e o estudo de algoritmos."},
    {"q": "o que e machine learning?", "a": "machine learning permite que sistemas aprendam padroes."}
]

def build_prompt_ids(question, answer=None, context_size=32):
    # Prompt compacto sem pontuacao rara no vocab
    prefix = "pergunta\n"
    suffix = "resposta\n"
    q_ids = encode(question)
    if answer is not None:
        # adiciona "\n" ao final para ensinar parada
        a_ids = encode(answer + "\n")
    else:
        a_ids = []
    base_ids = encode(prefix) + encode(suffix)

    # Prioriza manter a pergunta inteira; trunca a resposta se necessário
    max_a = max(0, context_size + 1 - len(base_ids) - len(q_ids))
    if max_a < len(a_ids):
        a_ids = a_ids[:max_a]

    # Se a pergunta for grande demais, trunca o início para caber
    max_q = max(1, context_size + 1 - len(base_ids) - len(a_ids))
    if len(q_ids) > max_q:
        q_ids = q_ids[-max_q:]

    cmd_ids = encode(prefix) + q_ids + encode(suffix)
    full_ids = cmd_ids + a_ids
    return cmd_ids, full_ids

def build_sft_dataset(data, context_size):
    X, Y, masks = [], [], []
    pad_id = stoi.get(' ', 0)
    for item in data:
        cmd_ids, full_ids = build_prompt_ids(item['q'], item['a'], context_size)
        cmd_len = len(cmd_ids)
        if len(full_ids) < 2:
            continue
        if len(full_ids) > context_size + 1:
            full_ids = full_ids[: context_size + 1]
            cmd_len = min(cmd_len, len(full_ids))
        x = full_ids[:-1]
        y = full_ids[1:]
        if cmd_len >= len(x):
            continue
        # Máscara: 0 no comando/padding, 1 na resposta
        m = [1 if (i + 1) >= cmd_len else 0 for i in range(len(x))]
        if len(x) < context_size:
            pad_len = context_size - len(x)
            x = x + [pad_id] * pad_len
            y = y + [pad_id] * pad_len
            m = m + [0] * pad_len
        X.append(x)
        Y.append(y)
        masks.append(m)
    return torch.tensor(X).to(device), torch.tensor(Y).to(device), torch.tensor(masks).to(device)

X, Y, M = build_sft_dataset(instructions, context_size)
print(f"📦 Amostras de Alinhamento: {len(X)}")


## 2. Treinamento com Máscara de Loss

Otimizamos apenas a geração da resposta.

![Masking](./infograficos/03-mascaramento-loss-resposta.png)

In [ ]:
model = GPTMini(config).to(device)
model.load_state_dict(ckpt['state_dict'])
# Desliga dropout para memorizar o pequeno dataset
for m in model.modules():
    if isinstance(m, nn.Dropout):
        m.p = 0.0
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.0)

loss_history = []
model.train()
print("🔨 Alinhando o assistente...")
batch_size = len(X)
for step in range(2001):
    idx = torch.randint(len(X), (batch_size,))
    logits, _ = model(X[idx])
    B, T, V = logits.shape
    loss = F.cross_entropy(logits.view(-1, V), Y[idx].view(-1), reduction='none')
    mask = M[idx].view(-1)
    loss = (loss * mask).sum() / mask.sum().clamp(min=1)
    optimizer.zero_grad(set_to_none=True); loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    loss_history.append(loss.item())
    if step % 500 == 0: print(f"Step {step} | Loss {loss.item():.4f}")

plt.plot(loss_history, color='#34A853')
plt.title("Curva de Alinhamento (SFT)")
plt.show()


## 3. Teste do Assistente Alinhado

O modelo agora responde apenas o que foi solicitado.

In [ ]:
@torch.no_grad()
def ask(model, question, max_new_tokens=80, no_repeat_ngram=3):
    model.eval()
    cmd_ids, _ = build_prompt_ids(question, answer=None, context_size=context_size)
    idx = torch.tensor(cmd_ids).unsqueeze(0).to(device)
    prompt_len = len(cmd_ids)

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        logits, _ = model(idx_cond)
        # no-repeat ngram (n=3)
        if no_repeat_ngram and idx.shape[1] >= no_repeat_ngram - 1:
            n = no_repeat_ngram
            prefix = tuple(idx[0, -(n-1):].tolist())
            seen = set()
            seq = idx[0].tolist()
            for i in range(len(seq) - n + 1):
                seen.add(tuple(seq[i:i+n]))
            for cand in range(logits.shape[-1]):
                if prefix + (cand,) in seen:
                    logits[0, -1, cand] = -float('inf')
        next_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        idx = torch.cat([idx, next_id], dim=1)
        if itos[next_id.item()] in ['\n', '.']:
            break

    # Retornamos apenas a parte gerada (Resposta)
    return decode(idx[0][prompt_len:].tolist())

print("🤖 TESTE DE INTERAÇÃO:")
print("-" * 30)
q1 = "o que o gato fez?"
print(f"Pergunta: {q1}\nResposta: {ask(model, q1)}")

print("\n" + "-" * 30)
q2 = "o que e machine learning?"
print(f"Pergunta: {q2}\nResposta: {ask(model, q2)}")

print("\n" + "-" * 30)
q3 = "defina inteligencia artificial"
print(f"Pergunta: {q3}\nResposta: {ask(model, q3)}")

print("\n" + "-" * 30)
q4 = "onde o cachorro dormiu?"
print(f"Pergunta: {q4}\nResposta: {ask(model, q4)}")


## 🏁 Conclusão da Jornada

Você completou a série! 

Transformou um modelo estatístico em um assistente capaz de seguir intenções humanas. Este é o fundamento do alinhamento de IA.

![Avaliação](./infograficos/05-avaliacao-respostas.png)